# Setup

In [1]:
import duckdb
from splink import DuckDBAPI, Linker, SettingsCreator, block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
    count_comparisons_from_blocking_rule,
    n_largest_blocks,
)

In [2]:
con = duckdb.connect()

df_l = con.read_parquet("./perturbed_data_l.parquet")
df_r = con.read_parquet("./perturbed_data_r.parquet")

db_api = DuckDBAPI(connection=con)

# Basic blocking

In [3]:
brs = [
    block_on("fname", "substring(sname, 1, 2)"),
    block_on("dob"),
    block_on("address", arrays_to_explode=["address"]),
    block_on("substr(fname, 1, 2)", "year(dob)"),
    block_on("year(dob)", "sname"),
]

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[df_l, df_r],
    blocking_rules=brs,
    link_type='link_only',
    db_api=db_api,
)

alt.Chart(...)

In [4]:
n_largest_blocks(
    table_or_tables=[df_l, df_r],
    blocking_rule=block_on("fname"),
    link_type='link_only',
    db_api=db_api,
    n_largest=10,
).as_pandas_dataframe()

,key_0,count_l,count_r,block_count
0,HSTG,1605,1631,2617755
1,SBYWSH,1577,1653,2606781
2,VTNAET,1586,1562,2477332
3,CUQDU,1557,1554,2419578
4,OUOYNT,1551,1520,2357520
5,RGQRXJ,1483,1492,2212636
6,VONZZK,1422,1424,2024928
7,XLSAUBK,1430,1403,2006290
8,XZUGDGW,1350,1404,1895400
9,KOSEA,1378,1371,1889238


# Advanced blocking - maximum block size

Sometimes a field is very skewed - a few values have a very high frequency, while most values have a relatively low frequency. Names are often like this. It may be useful to block on these fields, but only using the values with a low frequency. 

If you do this, some records will be excluded because their value for that field is too common. This means that you need to make sure there are other blocking rules that can pick up potential links for those records. The following example shows this by blocking on first names that appear less than 500 times, and following that by blocking on first name and surname (with no restriction on frequency). 

In [5]:
df_concat = con.sql(
    """
    select *, 'df_l' as source_dataset
    from df_l 
    
    union all
    
    select *, 'df_r' as source_dataset
    from df_r
    """
)

# add a 
fn_counts = con.sql(
    """
    select
        fname,
        case
            when count(*) < 500 then fname
            else NULL
        end as fn_rare
    from df_concat
    group by fname
    """
)

df = con.sql(
    """
    select *
    from df_concat
    left join fn_counts
    using (fname)
    """
)

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[df],
    blocking_rules=[block_on("fn_rare"), block_on("fname", "sname")],
    link_type='link_only',
    db_api=db_api,
)

alt.Chart(...)

This approach limits the number of comparisons for fname to 1,600,000, while there are an additional 300,000 record pairs that share both first name and surname, but were excluded from the first blocking rule. 

These two rules combined are much lower than just blocking on fname, which generates about 67,000,000 comparisons.


# Assessing blocking rules

Above, we have focused on counting how many comparisons the blocking rules generate, but the other major consideration is how many true matches are captured by the blocking rules. We have no way of knowing this for sure, because we don't know what the true matches are. 

One way to roughly approximate the coverage of a blocking rule is to look at the size of the blocks for each record. Some of these blocks will have size 1 - so that record won't be compared with anything else. You can extend this by looking at the maximum block size for that record across all of the blocking rules. This means that we can roughly see the overall coverage for a set of blocking rules.

In [6]:
counts_1 = con.sql(
    """
    select
        fname,
        sname,
        count(*) as n
    from df_concat
    where fname is not null and sname is not null
    group by fname, sname
    """
)

df_with_counts_1 = con.sql(
    """
    select 
        df_concat.unique_id,
        coalesce(n, 0) as n
    from df_concat
    left join counts_1
    using (fname, sname)
    """
)

counts_2 = con.sql(
    """
    select
        dob,
        count(*) as n
    from df_concat
    where dob is not null
    group by dob
    """
)

df_with_counts_2 = con.sql(
    """
    select
        df_concat.unique_id,
        coalesce(n, 0) as n
    from df_concat
    left join counts_2
    using (dob)
    """
)

max_block_sizes = con.sql(
    """
    select
        unique_id,
        max(n) as n
    from (
        select * 
        from df_with_counts_1

        union all

        select *
        from df_with_counts_2
    )
    group by unique_id
    """
)

con.sql(
    """
    select count(*)
    from max_block_sizes
    where n <= 1
    """
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            3 │
└──────────────┘

# Advanced blocking - Locality Sensitive Hashing

Locality Sensitive Hashing (LSH) is a technique for randomly placing records into blocks so that records that are more similar are more likely to be placed in the same block. This is a more advanced technique to implement, but it is a more general approach to blocking than selecting blocking rules by hand.

The idea behind LSH is to split each record into a collection of q-grams (overlapping strings of length q). The similarity of two records is measured using the Jaccard similarity of the two sets of q-grams, which is the size of their intersection divided by the size of their union. For each record, some number of qgrams are selected using min-hashes (see resources for more information). These are divided up into b bands of r qgrams. Each record is placed into b blocks, and each block of those blocks is the set of records that agree on those r qgrams. 
The numbers b and r are the two main parameters for LSH. Increasing b means that records have a higher chance to be placed in the same block, and increasing r means that two records have to be more similar to be placed into the same block. 

LSH has some of the same problems as selecting blocking rules by hand. You can still get some high frequency values that create very large blocks, which generates a lot of comparisons. LSH can be combined with techniques that filter out high-frequency blocks (or rank based filtering as used in the first resource).

LSH is not implemented in Splink.

**Resources**

- [Locality Sensitive Hashing with Temporal and Spatial Constraints for
Efficient Population Record Linkage](https://dl.acm.org/doi/pdf/10.1145/3511808.3557631)
- [UNIQUE ENTITY ESTIMATION WITH APPLICATION TO THE SYRIAN CONFLICT](https://www.jstor.org/stable/26542562)
- [Blocking and Filtering Techniques for Entity Resolution:
A Survey](https://dl.acm.org/doi/pdf/10.1145/3377455)



In [7]:
# very rough implementation to demonstrate LSH
def create_q_grams(connection, tbl, blocking_cols, id_column, q):
    select_blocking_cols = ", ".join(blocking_cols)
    
    tbl_with_blocking_string = connection.sql(
        f"""
        select 
            {id_column},
            concat_ws('_', {select_blocking_cols}) as blocking_string
        from tbl
        """
    )

    tbl_with_qgrams = connection.sql(
        f"""
        select
            {id_column},
            list_transform(
                range(0, length(blocking_string) - {q} + 1),
                lambda i : blocking_string[i+1 : i+{q}]
            ) as qgrams
        from tbl_with_blocking_string
        """
    )
    
    return tbl_with_qgrams


def create_lsh_blocks(connection, tbl, blocking_cols, id_column, q, b, r):
    """
    Inputs:
        - connection: the DuckDB connection object
        - tbl: the input table as a DuckDB relation object (or a pandas dataframe)
        - blocking_cols: a list of column names to use in the LSH steps
        - id_column: the name of the column that uniquely identifies the units in the input table
        - q: the number q for which to create q-grams (e.g. 3-grams would be triples of letters 'ABC' or 'XYZ')
        - b: the number of bands for LSH
        - r: the number of rows for LSH

    Returns:
        A DuckDB relation with the id_column and min_hash columns.
    """
    qgrams = create_q_grams(
        connection=connection,
        tbl=tbl,
        blocking_cols=blocking_cols,
        id_column=id_column,
        q=q,
    )

    min_hash_exprs = [
        f"list_min(list_transform(qgrams, lambda x : hash(x, {i}))) as min_hash_{i}"
        for i in range(b * r)
    ]

    select_hashes = ", ".join(min_hash_exprs)

    min_hashes = connection.sql(
        f"""
        select
            {id_column},
            {select_hashes}
        from qgrams
        """
    )

    return min_hashes

In [8]:
create_q_grams(con, df, ["fname", "sname", "strftime(dob, '%y%m%d')"], "unique_id", q=3)

┌───────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ unique_id │                                                     qgrams                                                     │
│   int64   │                                                   varchar[]                                                    │
├───────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│         0 │ [ZRB, RBA, BA_, A_B, _BR, BRJ, RJT, JTJ, TJZ, JZC, ZCQ, CQE, QE_, E_0, _06, 061, 611, 110, 103]                │
│         1 │ [AEP, EPM, PMP, MPP, PP_, P_C, _CM, CMU, MUO, UOG, OGR, GRD, RDQ, DQS, QSK, SK_, K_0, _04, 040, 405, 052, 520] │
│         2 │ [ADC, DCH, CHR, HRJ, RJ_, J_I, _IH, IHZ, HZV, ZVY, VYS, YSU, SUQ, UQ_, Q_4, _45, 450, 506, 062, 620]           │
│         3 │ [LOP, OPF, PFU, FUH, UH_, H_M, _MD, MDM, DMU, MUB, UBD, BDW, DWB, WB_, B_6, _61, 610, 108, 082, 8

In [9]:
con.sql("drop table if exists blocks")
blocks = create_lsh_blocks(con, df, ["fname", "sname", "strftime(dob, '%y%m%d')"], "unique_id", q=3, b=4, r=2)
# create a pandas dataframe from the DuckDB result
blocks = blocks.to_df()

In [10]:
def erase_large_blocks(connection, tbl, blocking_cols, max_block_size=1000):
    cols_expr = ", ".join(blocking_cols)
    erased_cols = [
        f"case when count(*) < {max_block_size} then {bc} else NULL end as new_{bc}"
        for bc in blocking_cols
    ]

    erased_cols_expr = ", ".join(erased_cols)

    erased = connection.sql(
        f"""
        select
            {cols_expr},
            {erased_cols_expr}
        from tbl
        group by {cols_expr}
        """
    )

    blocks_erased = connection.sql(
        f"""
        select *
        from tbl
        left join erased
        using ({cols_expr})
        """
    )

    return blocks_erased

In [11]:
df_new = erase_large_blocks(con, blocks, ["min_hash_0", "min_hash_1"], max_block_size=1000).to_df()
df_new = erase_large_blocks(con, df_new, ["min_hash_2", "min_hash_3"], max_block_size=1000).to_df()
df_new = erase_large_blocks(con, df_new, ["min_hash_4", "min_hash_5"], max_block_size=1000).to_df()
df_new = erase_large_blocks(con, df_new, ["min_hash_6", "min_hash_7"], max_block_size=1000).to_df()

In [12]:
df_new

,unique_id,min_hash_0,min_hash_1,min_hash_2,min_hash_3,min_hash_4,min_hash_5,min_hash_6,min_hash_7,new_min_hash_0,new_min_hash_1,new_min_hash_2,new_min_hash_3,new_min_hash_4,new_min_hash_5,new_min_hash_6,new_min_hash_7
0,39511,2117826630078639663,1809535619390962245,143388587828243581,934317711088741326,746304373771370484,877793443619369231,472118703260493440,354418757258572019,2117826630078639663,1809535619390962245,143388587828243581,934317711088741326,746304373771370484,877793443619369231,472118703260493440,354418757258572019
1,39512,331040368134670217,644719710618303846,924230409780129992,1113168210802869866,711285388107518544,2082346788535613863,356575713913192085,474187711500691686,331040368134670217,644719710618303846,924230409780129992,1113168210802869866,711285388107518544,2082346788535613863,356575713913192085,474187711500691686
2,39513,252279265314962282,285536634243968335,746078761813827163,2301903491309661936,1827964307229702858,2209757870890450283,1473251197128562191,1591005035927257212,252279265314962282,285536634243968335,746078761813827163,2301903491309661936,1827964307229702858,2209757870890450283,1473251197128562191,1591005035927257212
3,39514,927815714158924103,2476037496985440678,925028743981003111,1440288677726889904,1351774636691101272,1545060159227651775,1764537784511995727,1880971147393009980,927815714158924103,2476037496985440678,925028743981003111,1440288677726889904,1351774636691101272,1545060159227651775,1764537784511995727,1880971147393009980
4,39515,60540784878755556,1278833973946334890,710694373173830589,275100578417442253,95376000206976569,342397202965872387,599350021395662130,736119898108875585,60540784878755556,1278833973946334890,710694373173830589,275100578417442253,95376000206976569,342397202965872387,599350021395662130,736119898108875585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185395,39506,1984735328824887397,276556267706021762,10581799100748292,113994728836267098,520384867649185888,620064644814272368,186801362214816266,68062344757288057,1984735328824887397,276556267706021762,10581799100748292,113994728836267098,520384867649185888,620064644814272368,<NA>,<NA>
185396,39507,914225350386473526,2507127029195112981,1168288470672250980,148213173846002152,271222543794405238,173668688853056254,688633166225076503,858196011463272292,914225350386473526,2507127029195112981,1168288470672250980,148213173846002152,271222543794405238,173668688853056254,688633166225076503,858196011463272292
185397,39508,914225350386473526,285536634243968335,341753608426881070,638192782333344567,1042418567129657101,560260122559205970,241888004208608200,89284009288357307,914225350386473526,285536634243968335,341753608426881070,638192782333344567,1042418567129657101,560260122559205970,241888004208608200,89284009288357307
185398,39509,109810173821264718,1429736745385425795,509559267715271223,1007554031815589776,629177363391108732,353749230657575369,402923218171886265,296655575906930972,109810173821264718,1429736745385425795,509559267715271223,1007554031815589776,629177363391108732,353749230657575369,402923218171886265,296655575906930972
